In [9]:
import pandas as pd
import polars as pl
from pathlib import Path
from download_and_preprocessing import Preprocessor, Metadata
from pprint import pprint
import numpy as np
from loguru import logger
from time import time
from gensim.models import ensemblelda
from gensim.models import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import os

paths = {
        "metadata": Path(r"data\export_datamart.csv"),
        "all_speeches": Path(r"data\all_ECB_speeches.csv"),
        "html_dir": Path(r"data\speeches"),
        "parsed_file": Path(r"data\speeches\parsed.jsonl"),
        "preprocessed_dir": Path(r"data\preprocessed_en"),
    }

preprocessed_data, dictionary, bow_corpus, tfidf_corpus = Preprocessor().load_df_dict_corpus(paths["preprocessed_dir"])

In [38]:
preprocessed_data

,content,related_topics,id,date,author,title,url,language,type_long,tokens
0,"Lucas Papademos, Vice-President of the ECB Jej...",[],1000,2008-06-16,Lucas Papademos,Statement at the 2008 Asia-Europe (ASEM) Finan...,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[lucas, papademos, vice, president, ecb, jeju,..."
1,"Speech by Gertrude Tumpel-Gugerell, Member of ...",[],1002,2008-06-18,Gertrude Tumpel-Gugerell,Towards an integrated securities market - The ...,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[speech, gertrude, tumpel, gugerell, member, e..."
2,"Lorenzo Bini Smaghi, Member of the Executive B...",[],1005,2008-06-24,Lorenzo Bini Smaghi,Public Hearing on EMU@10: The first ten years ...,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[lorenzo, bini, smaghi, member, executive, boa..."
3,"Introductory statement by Jean-Claude Trichet,...",[],1006,2008-06-25,Jean-Claude Trichet,Hearing at the Economic and Monetary Affairs C...,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[introductory, statement, jean, claude, triche..."
4,"Speech by Lucas Papademos, Vice-President of t...",[],1007,2008-06-26,Lucas Papademos,Financial market excesses and corrections: a c...,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[speech, lucas, papademos, vice, president, ec..."
...,...,...,...,...,...,...,...,...,...,...
3697,"Speech by Jürgen Stark, Member of the Executiv...",[],996,2008-06-11,Jurgen Stark,Lessons for central bankers from the history o...,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[speech, jürgen, stark, member, executive, boa..."
3698,"Lorenzo Bini Smaghi, Member of the Executive B...",[],997,2008-06-12,Lorenzo Bini Smaghi,Supervision and central banking: improving the...,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[lorenzo, bini, smaghi, member, executive, boa..."
3699,"Gertrude Tumpel-Gugerell, Member of the Execut...",[],999,2008-06-16,Gertrude Tumpel-Gugerell,Speech at the International Bankers' Club,https://www.ecb.europa.eu/press/key/date/2008/...,EN,speech,"[gertrude, tumpel, gugerell, member, executive..."
3700,Speech by Professor Otmar Issing Member of the...,[],99,1999-05-07,Otmar Issing,The euro area economy on its way to the new mi...,https://www.ecb.europa.eu/press/key/date/1999/...,EN,speech,"[speech, professor, otmar, issing, member, exe..."


In [39]:
metadata = Metadata().metadata_df
print(metadata.shape)
authors = list(set([y for x in metadata['who'].str.split(',').to_list() for y in x]))
cleaned_authors = []
for name in authors: 
    if ' and ' in name:
        cleaned_names = list(name.split(' and '))
    else: 
        cleaned_names = [name]
    for n in cleaned_names:
        n = ''.join([i for i in n if not i.isdigit()]).replace('[', '').replace(']', '').strip()
        cleaned_authors.append(n)

cleaned_authors = list(set(cleaned_authors))
cleaned_authors

(3983, 10)


['Vice-President Luis de Guindos',
 'Cornelia Holthausen',
 'Vasco Botelho',
 'Kerstin af Jochnick',
 'Otmar Issing',
 'Siria Angino',
 'Massimo Ferrari Minesso',
 'Lorenz Emter',
 'Katie Ranger',
 'Martina Di Sano',
 'Manfred Kremer',
 'Stefan Gebauer',
 'Fabian Joshua Schmidt',
 'Óscar Arce',
 'Othman Bouabdallah',
 'Ulrich Bindseil',
 'Stefania Secola',
 'Julian Schumacher',
 'Werner Hoyer',
 'Jorg Asmussen',
 'Sarai Artiles Reyes',
 'Lara Vivian',
 'Sarah Holton',
 'Willem F. Duisenberg',
 'Paul Reimers',
 'Vanessa Gunnella',
 'Ewald Nowotny',
 'José Manuel Campa',
 'Paul Hiebert',
 'Lesetja Kganyago',
 'Elizabeth McCaul',
 'Marta Skrzypińska',
 'Simon Mee',
 'Johannes Gareis',
 'Benoit Nguyen',
 'Imène Rahmouni-Rousseau',
 'Joao Sousa',
 'Ramon Quintana Aguirre',
 'Friderike Kuik',
 'Till Folger',
 'Eva Taylor',
 'Paschal Donohoe',
 'Linette Field',
 'Johannes Breckenfelder',
 'Vilem Valenta',
 'Patricia Roa Tejero',
 'Elena Rancoita',
 'Tobias Linzert',
 'Mattia Ricci',
 'Satu Hi

In [ ]:
import gensim
text = "Vice-President Luis de Guindos speech at the European Banking Federation's European Banking Summit 2021"
text_l = text.lower()



# [x for x in gensim.utils.tokenize(text, lower=True)]